# Model Training
Welcome to the 'Model Training and Prediction' notebook, a crucial facet of our project's data science pipeline. In this notebook, we offer a meticulous examination of our rigorous model development process. The pipeline starts by accepting training data, followed by fitting three distinct types of models to it: Random Forest, Gradient Boosted Tree, and XGBoost. The initial stages include encoding categorical variables and executing Recursive Feature Elimination (RFE) for feature selection. This is succeeded by the application of genetic algorithms to hyperparameter tuning, operating in tandem with a cross-validation routine. Subsequently, the best model is selected based on the highest F1 score, indicating the balance between precision and recall. Finally, the selected model is utilized to predict the outcomes for the current week's round of NRL matches. This process is iterative and cyclical, with the potential for revisiting earlier stages based on the model's performance. Let us proceed with this in-depth exploration.

## Set up Environment
This code segment is setting up the environment for the model training pipeline. It begins by importing sys and pathlib - Python libraries used for managing system parameters and file paths, respectively.

The code then updates the system path to include the "functions" directory. This allows for the import of custom modules `modelling_functions`, `model_properties`, and `training_config` which are stored in this directory. These modules contain custom functions and configuration settings that are critical for the later stages of data preprocessing, model training, and prediction.

Following this, the `project_root` variable is defined. This is achieved by using the pathlib library to establish the root directory of the project.

Finally, the `db_path` variable is constructed. This is the relative path to the SQLite database "footy-tipper-db.sqlite", which is located in the "data" directory of the project root. This path will be used for database connectivity throughout the pipeline.

In [1]:
# import libraries
import os
import sys
import pathlib

cwd = os.getcwd()

# get the parent directory
parent_dir = os.path.dirname(cwd)

# add the parent directory to the system path
sys.path.insert(0, parent_dir)

# Get to the root directory
project_root = pathlib.Path().absolute().parent

# import functions from common like this:
from pipeline.common.model_training import (
    training_config as tc,
    modelling_functions as mf,
    model_properties as mp
)

from pipeline.common.model_prediciton import prediction_functions as pf

## Get data
Our process starts by establishing the root directory of the project and constructing the relative path to the 'footy-tipper-db.sqlite' database located within the 'data' directory. We then connect to this SQLite database and use a SQL query housed in the 'footy_tipping_data.sql' file, found in the 'sql' directory, to extract the required data. This data is loaded into a pandas DataFrame, footy_tipping_data, serving as the basis for our subsequent modeling activities. Upon successful extraction of the data, we ensure the database connection is closed, maintaining good coding practice and resource management.

In [2]:
data = mf.get_training_data(
    db_path = project_root / "data" / "footy-tipper-db.sqlite", 
    sql_file = project_root / 'pipeline/common/sql/training_data.sql')

data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,away_prev_result_diff,prev_result_diff,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.018111e+10,1.0,Round 1,1.0,Final,1.520540e+09,1.520500e+09,Netstrata Jubilee Stadium,Sydney,14457.0,...,0.0,0.0,1498.872125,1510.437715,-11.565590,0.476017,0.500844,0.023139,-0.024826,2.595637
1,2.018111e+10,1.0,Round 1,2.0,Final,1.520618e+09,1.520579e+09,McDonald Jones Stadium,Newcastle,23516.0,...,0.0,0.0,1484.764385,1496.167942,-11.403557,0.476245,0.500616,0.023139,-0.024371,5.203137
2,2.018111e+10,1.0,Round 1,3.0,Final,1.520622e+09,1.520586e+09,1300SMILES Stadium,Townsville,15900.0,...,0.0,0.0,1506.234819,1503.247293,2.987526,0.496473,0.480388,0.023139,0.016085,1.723980
3,2.018111e+10,1.0,Round 1,4.0,Final,1.520699e+09,1.520660e+09,Accor Stadium,Sydney,18243.0,...,0.0,0.0,1493.847313,1500.969327,-7.122014,0.482262,0.494599,0.023139,-0.012337,-5.050546
4,2.018111e+10,1.0,Round 1,5.0,Final,1.520698e+09,1.520669e+09,Other,Perth,38824.0,...,0.0,0.0,1490.494449,1483.490433,7.004017,0.502117,0.474745,0.023139,0.027372,1.126557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,2.024111e+10,20.0,Round 20,3.0,Final,1.721497e+09,1.721461e+09,McDonald Jones Stadium,Newcastle,26952.0,...,16.0,-8.0,1476.162187,1496.481774,-20.319587,0.461550,0.510720,0.027730,-0.049170,-0.496762
1330,2.024111e+10,20.0,Round 20,4.0,Final,1.721504e+09,1.721468e+09,AAMI Park,Melbourne,17055.0,...,-10.0,20.0,1526.671267,1559.705708,-33.034441,0.443845,0.528425,0.027730,-0.084580,3.201997
1331,2.024111e+10,20.0,Round 20,5.0,Final,1.721570e+09,1.721534e+09,BlueBet Stadium,Penrith,20955.0,...,20.0,-30.0,1529.181065,1499.925781,29.255284,0.529463,0.440416,0.030120,0.089047,12.614410
1332,2.024111e+10,20.0,Round 20,6.0,Final,1.721578e+09,1.721542e+09,4 Pines Park,Sydney,16050.0,...,-4.0,42.0,1521.778668,1496.040295,25.738373,0.524592,0.445288,0.030120,0.079304,8.301481


In [6]:
data["competition_year"]

0       2018
1       2018
2       2018
3       2018
4       2018
        ... 
1329    2024
1330    2024
1331    2024
1332    2024
1333    2024
Name: competition_year, Length: 1334, dtype: int64

In [5]:
data.columns.to_list()

['game_id',
 'round_id',
 'round_name',
 'game_number',
 'game_state_name',
 'start_time',
 'start_time_utc',
 'venue_name',
 'city',
 'crowd',
 'broadcast_channel1',
 'broadcast_channel2',
 'broadcast_channel3',
 'team_home',
 'team_final_score_home',
 'team_head_to_head_odds_home',
 'team_line_odds_home',
 'team_line_amount_home',
 'team_away',
 'team_final_score_away',
 'team_head_to_head_odds_away',
 'team_line_odds_away',
 'team_line_amount_away',
 'competition_year',
 'position_home',
 'wins_home',
 'draws_home',
 'losses_home',
 'byes_home',
 'competition_points_home',
 'points_for_home',
 'points_against_home',
 'points_difference_home',
 'home_wins_home',
 'home_draws_home',
 'home_losses_home',
 'away_wins_home',
 'away_draws_home',
 'away_losses_home',
 'recent_form_home',
 'season_form_home',
 'tries_for_home',
 'tries_conceded_home',
 'goals_for_home',
 'goals_conceded_home',
 'field_goals_for_home',
 'field_goals_conceded_home',
 'players_used_home',
 'average_winning_mar

## Modelling

During the modeling phase, the `train_and_select_best_model` function, part of our `modelling_functions` module, is invoked. This function initiates the training of Poisson models specifically designed for predicting the scores of the home and away teams. It takes as input the footy tipping data, predictor variables, the outcome variable, and several configuration settings like whether to use Recursive Feature Elimination (RFE), the number of cross-validation folds, and the optimization metric, all sourced from the `training_config` module.

The function first identifies categorical columns in the feature set for one-hot encoding, creating dummy variables for categorical features. Depending on the choice of using RFE, a feature elimination step may be included in the pipeline. Each model subsequently undergoes hyperparameter tuning using a genetic algorithm, facilitated by the `GASearchCV` function.

All the models are then trained and evaluated through cross-validation. The best models for predicting home and away scores are selected based on superior performance on the chosen optimization metric. The selected models, encapsulated in pipelines with pre-processing steps and hyperparameter tuning, are now ready for the prediction phase.

The trained models are saved using the `save_models` function for future use, ensuring that the prediction process can be efficiently replicated and scaled.

### Test Modelling Data Preparation
First, because we want to understand how effective our simulations are without creating more load on the model cross-validation, we will split the data into training and testing sets. This will allow us to evaluate the model's interaction with each other on unseen data more easily.

In [3]:
# Set the random seed for reproducibility
random_seed = 42

# Define the test size proportion
test_size = 0.2

# Randomly shuffle the DataFrame and split
training_data = data.sample(frac=1 - test_size, random_state=random_seed)
test_data = data.drop(training_data.index)

### Poisson Modelling
The Poisson model is a statistical model that is used to predict the number of events occurring within a fixed interval of time or space. In the context of sports, the Poisson model can be used to predict the number of goals or points scored by each team in a match. In this section, we will implement a Poisson model to predict the number of points scored by each team in a match. We will then use these predictions to calculate the expected match outcome.

#### Home Modelling

In [4]:
home_model = mf.train_and_select_best_model(
    training_data, tc.predictors, 'team_final_score_home',
    tc.use_rfe, tc.num_folds, tc.opt_metric
)


Model training: LGBMRegressor


KeyError: "['sin_bin_home', 'conversion_made_home', 'conversion_missed_home', 'tackle_made_home', 'tackle_missed_home', 'possession_home', 'territory_home', 'offloads_home', 'tackle_break_home', 'linebreak_home', 'lb_assist_home', 'kicks_home', 'kicks_occur_home', 'try_assist_home', 'error_home', 'try_home', 'penalty_shot_made_home', 'penalty_shot_missed_home', 'field_goal_made_home', 'field_goal_missed_home', 'points_home', 'kick_return_home', 'dh_run_home', 'dh_run_occur_home', 'x40_20_kick_home', 'kick_bomb_home', 'try_assists_and_involvements_home', 'line_break_assists_and_involvements_home', 'shifts_home', 'shortsides_home', 'captains_challenge_success_percent_home', 'x20_att_gl_possession_seconds_home', 'marker_tackle_home', 'trebles_home', 'markers_home', 'shortside_left_home', 'shortside_right_home', 'charge_downs_home', 'kick_charged_down_home', 'decoys_home', 'ineffective_tackle_home', 'intercept_home', 'pass_intercepted_home', 'one_on_one_steal_home', 'on_report_home', 'one_

##### Feature Imprtance

In [ ]:
preprocessor = home_model.named_steps['one_hot_encoder']
encoded_feature_names = preprocessor.get_feature_names_out(tc.predictors)

# Get the top 20 feature importances
top_20_features = mp.get_feature_importances(home_model, encoded_feature_names)
print(top_20_features)

#### Away Team Modelling

In [ ]:
away_model = mf.train_and_select_best_model(
    training_data, tc.predictors, 'team_final_score_away',
    tc.use_rfe, tc.num_folds, tc.opt_metric
)

##### Feature Importance

In [ ]:
# Get the names of the features after preprocessing (one-hot encoding)
preprocessor = away_model.named_steps['one_hot_encoder']
encoded_feature_names = preprocessor.get_feature_names_out(tc.predictors)

# Get the top 20 feature importances
top_20_features = mp.get_feature_importances(away_model, encoded_feature_names)
print(top_20_features)

### Example Match Simulation
In this section, we will simulate a match between two teams using the Poisson model. We will generate the expected number of points scored by each team and use these predictions to determine the match outcome. We will then compare the predicted outcome with the actual outcome to evaluate the accuracy of the Poisson model.

In [ ]:
# Example usage with test_data
mp.plot_sampling_distributions(home_model, away_model, test_data, tc.predictors)

###  Evaluation
The evaluation phase involves predicting the outcomes of an unseen set of random NRL matches using the selected home and away models.

This section is only to be used in development and testing at the moment.
#### Classification Report

In [ ]:
# Evaluate the models on the test data
result_df = mp.evaluate_models(home_model, away_model, test_data, tc.predictors)

#### Expected Scores

In [ ]:
mp.evaluate_score_predictions(result_df)

## Save Models
The `save_models` function stores Pipeline objects to the disk. This allows for easy retrieval and reuse in future model prediction tasks, without the need to retrain these components. The objects are stored in a designated 'models' directory under the project root path, ensuring organized and consistent storage.

In [ ]:
mf.save_models(home_model, 'home_model', project_root)
mf.save_models(away_model, 'away_model', project_root)

## Match Simulation and Prediction
The final stage of the pipeline involves predicting the outcomes of the current week's NRL matches. This is achieved by loading the saved models from the 'models' directory and utilizing them to simulate the matches. The predictions are then stored in a DataFrame, which is subsequently written back to the SQLite database. This data can be accessed by the front-end application to display the predicted outcomes to the users.

In [ ]:
# Load the models
home_model = pf.load_models('home_model', project_root)
away_model = pf.load_models('away_model', project_root)

# Load this week's game data
inference_data = pf.get_inference_data(
    db_path = project_root / "data" / "footy-tipper-db.sqlite", 
    sql_file = project_root / 'pipeline/common/sql/inference_data.sql')
inference_data

### Prediction Simulation
The `simulate_predictions` function is used to predict the outcomes of the current week's NRL matches. It loads the saved models from the 'models' directory and uses them to simulate the matches. The predictions are then stored in a DataFrame, which is written back to the SQLite database.

In [ ]:
import pandas as pd

# Predict match outcomes and scorelines for the inference data
outcomes, margins = pf.predict_match_outcome_and_scoreline_with_bayes(home_model, away_model, inference_data, tc.predictors)
outcome_df = pd.merge(outcomes, margins, on='game_id')
outcome_df

In [ ]:
# Save the predictions to the database
pf.save_predictions_to_db(
    outcome_df, 
    project_root / "data" / "footy-tipper-db.sqlite", 
    project_root / 'pipeline/common/sql/create_table.sql', 
    project_root / 'pipeline/common/sql/insert_into_table.sql'
)

## Sending Predictions via Email using ChatGPT
In this section, we will use the OpenAI ChatGPT model to generate an email template for sending the predictions to the users. We will use the predictions generated in the previous section and the ChatGPT model to create a personalized email template for each user. The email template will contain the predicted outcomes of the NRL matches for the current week.

In [ ]:
from dotenv import load_dotenv
from pipeline.common.model_prediciton import prediction_functions as pf
from pipeline.common.use_predictions import sending_functions as sf

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"
secrets_path = project_root / "secrets.env"
json_path = project_root / "service-account-token.json"

load_dotenv(dotenv_path=secrets_path)

In [ ]:
import sqlite3
import pandas as pd
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open(project_root / 'pipeline/common' / 'sql/prediction_table.sql', 'r') as file:
    query = file.read()

# Execute the query and fetch the results into a data frame
predictions = pd.read_sql_query(query, con)

# Disconnect from the SQLite database
con.close()

predictions

### Tippper Picks
The Tipper Picks are selcted games which show high value for the tipper to select. This is based on the predicted outcomes of the matches and the odds of the games.

This shows how predictions can be used to further enrich the user experience and provide valuable insights to the users.

In [ ]:
tipper_picks = sf.get_tipper_picks(predictions)
tipper_picks

In [ ]:
# sf.upload_df_to_drive(
#     predictions, 
#     json_path, 
#     os.getenv('FOLDER_ID'), 
#     "predictions.csv"
# )

### Reg Reagan's Email
In this section, we will generate an email from Reg Reagan, a fictional character, using the OpenAI ChatGPT model. The email will contain the predicted outcomes of the NRL matches for the current week, along with some humorous and engaging content. The email will be sent to the users to provide them with the predictions and entertain them at the same time.

In [ ]:
reg_reagan = sf.generate_reg_regan_email(
    predictions, 
    tipper_picks, 
    os.getenv('OPENAI_KEY'), 
    os.getenv('FOLDER_URL'),
    1
)

print(reg_reagan)

In [ ]:
# sf.send_emails(
#     "footy-tipper-email-list", 
#     f"Footy Tipper Predictions for {predictions['round_name'].unique()[0]}", 
#     reg_reagan, 
#     os.getenv('MY_EMAIL'), 
#     os.getenv('EMAIL_PASSWORD'), 
#     json_path
# )